In [1]:
# Parser
from os import listdir
from os.path import isfile, join

path_project_public_1 = './project_public_1/'

files_project_public_1 = [join(path_project_public_1, f) for f in listdir(path_project_public_1) if isfile(join(path_project_public_1, f))]

print(files_project_public_1)

['./project_public_1/bdbc_model.xml', './project_public_1/h264_interactions.txt', './project_public_1/h264_example_activation.txt', './project_public_1/bdbc_feature.txt', './project_public_1/h264.xml', './project_public_1/bdbc_interactions.txt', './project_public_1/h264_feature.txt']


In [64]:
bdbc_files = 'bdbc'
h264 = 'h264'
xml_files = '.xml'

var_map = []

final_string = ""
variables_string = ""
p_line = "p cnf "
clauses = ""
id_of_elememt = 0
current_name = ""
current_options = []
clauses_counter = 0
optional_false_elements = []
clause_clauses = []

for element in files_project_public_1:
    if h264 in element and xml_files in element:
    #if bdbc_files in element and xml_files in element:
        
        # capsule from here with an path to the .xml file to generate a dimacs string 
        # (input: 'element', output: 'final_string')
        
        from lxml import etree
        xml_rules = etree.parse(open(element, 'r'))
        root = xml_rules.getroot()
        for child in root :
            for subchild in child :
                for element in subchild : 
                    if element.tag == 'name':
                        var_map.append(element.text)
        
        for el in var_map:
            variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"
        
        for child in root :
            for subchild in child :
                for element in subchild :
                    if element.tag == 'name':
                        current_name = element.text
                    if element.tag == 'excludedOptions' :
                        for option in element :
                            current_options.append(option.text)
                    if element.tag == 'optional' :
                        if element.text != 'False' :
                            current_name = ''
                            current_options = []
                if current_name != '' :
                    optional_false_elements.append(1+var_map.index(current_name))
                    for item in current_options :
                        optional_false_elements.append((1+var_map.index(item))*(-1))
                    clause_clauses.append(optional_false_elements)
                    optional_false_elements = []
                    
                    optional_false_elements.append(1+var_map.index(current_name))
                    for item in current_options :
                        optional_false_elements.append(1+var_map.index(item))
                    optional_false_elements.sort()
                    if(clause_clauses.count(optional_false_elements) == 0) :
                        clause_clauses.append(optional_false_elements)
                    optional_false_elements=[]
                    
                    optional_false_elements.append((1+var_map.index(current_name))*(-1))
                    for item in current_options :
                        optional_false_elements.append((1+var_map.index(item))*(-1))
                    optional_false_elements.sort()
                    if(clause_clauses.count(optional_false_elements) == 0 and len(optional_false_elements)>1) :
                        clause_clauses.append(optional_false_elements)
                    optional_false_elements=[]
                    
                current_options = []
                                                        
        p_line += str(len(var_map))+" "+str(len(clause_clauses))+"\n"
        
        for line in clause_clauses :
            for item in line :
                clauses += str(item)+" "
            clauses += "0\n"
        
# und zack fertig .... dimacs
        final_string = variables_string+p_line+clauses
        print(final_string)
        print("\n")





c 1 no_asm
c 2 no_8x8dct
c 3 no_cabac
c 4 no_deblock
c 5 no_fast_pskip
c 6 no_mbtree
c 7 no_mixed_refs
c 8 no_weightb
c 9 rc_lookahead
c 10 rc_lookahead_20
c 11 rc_lookahead_40
c 12 rc_lookahead_60
c 13 ref
c 14 ref_1
c 15 ref_5
c 16 ref_9
p cnf 16 13
1 0
9 0
10 -11 -12 0
10 11 12 0
-12 -11 -10 0
11 -10 -12 0
12 -10 -11 0
13 0
14 -15 -16 0
14 15 16 0
-16 -15 -14 0
15 -14 -16 0
16 -14 -15 0



